# Pesquisar Queries

## Installs

## Imports

In [34]:
from tqdm import tqdm
import requests
import time
import re
import os
import io
import matplotlib.pyplot as plt

## Obter queries

In [1]:
!mkdir data
%cd data
!mkdir robust04
%cd robust04

!wget https://raw.githubusercontent.com/castorini/anserini/master/src/main/resources/topics-and-qrels/topics.robust04.txt
%cd ../../

/content/data
/content/data/robust04
--2021-12-09 01:39:02--  https://raw.githubusercontent.com/castorini/anserini/master/src/main/resources/topics-and-qrels/topics.robust04.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116975 (114K) [text/plain]
Saving to: ‘topics.robust04.txt’

topics.robust04.txt 100%[===================>] 114.23K  --.-KB/s    in 0.02s   

2021-12-09 01:39:02 (5.33 MB/s) - ‘topics.robust04.txt’ saved [116975/116975]

/content


## Função para extrair queries



In [55]:
def load_queries(path):
    print('Loading queries...')
    corpus = {}
    n_text = 0
    with open(path, errors='ignore') as f:
        all_text = f.read()
        for raw_text in tqdm(all_text.split('</top>')):
            # print(raw_text)
            if not raw_text:
                continue
            # result = re.search(r'\<num>(.*)er', raw_text)
            result = re.findall('[0-9]+', raw_text)
            # print(result)
            # break
            if not result:
                continue
            doc_id = result[0]            
            # doc_id = doc_id.strip()
            # print(doc_id)
            raw_text = ' '.join(raw_text.split())

            result = re.search(r'\<title>(.*)\<desc>', raw_text)
            doc_text = ''
            if result:
                doc_text = result.group(1)
                doc_text = doc_text.strip()
                if doc_text:
                    n_text += 1
                    
            corpus[doc_id] = ' '.join(doc_text.split())
        
    print(f'Loaded {len(corpus)} docs, {n_text} with texts.')
    return corpus

In [33]:
corpus = load_queries('/content/data/robust04/topics.robust04.txt')

Loading queries...


100%|██████████| 251/251 [00:00<00:00, 20267.41it/s]

Loaded 250 docs, 250 with texts.


## Testes

In [27]:
corpus.keys()

dict_keys(['301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '354', '355', '356', '357', '358', '359', '360', '361', '362', '363', '364', '365', '366', '367', '368', '369', '370', '371', '372', '373', '374', '375', '376', '377', '378', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '400', '401', '402', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '423', '424', '425', '426', '427', '428', '429', '430', '431', '432', '433', '434', '435', '436', '437', '438', '439', '440', '441', '4

In [29]:
corpus['301']

'Poliomyelitis and Post-Polio'

## Subscription Key

In [35]:
subscription_key = "bac181b9c2734bdeaf0b2f4748eea0e6"
assert subscription_key

## Obtendo documentos

In [41]:
def documents_for_querie(querie):

  search_url = "https://api.bing.microsoft.com/v7.0/search"

  search_term = "site:https://storage.googleapis.com/robust04-public " + querie

  all_urls = []

  for j in range(10):

    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    params = {"q": search_term, "textDecorations": False, "textFormat": "HTML", "count":50, "offset":j*10}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    time.sleep(1)

    for i in search_results['webPages']["value"]:

      result = re.search(r'public/(.*).pdf', i['url'])
      m = result.group(1)    

      if m not in all_urls:
        all_urls.append(m)

    if len(all_urls) >= search_results['webPages']['totalEstimatedMatches']:
      print(f"Obteve todos na iteração {j+1}")
      break

  return all_urls

In [45]:
all_urls = documents_for_querie(" ")

urls_encontradas, remover = [], []

for i in all_urls:
  if len(i) == 11 and i.isalpha():
    urls_encontradas.append(i)
  else:
    remover.append(i)

print(f'URLs encontradas:  {len(urls_encontradas)}')
print(f'URLs para remover: {len(remover)}')

Obteve todos na iteração 4
URLs encontradas:  71
URLs para remover: 9


In [54]:
relevantes = {}
for j in corpus.keys():
  URLS = []
  URLS.append(documents_for_querie(corpus[j]))

  relevantes[i] = URLS

Obteve todos na iteração 4
